# Imports

In [1]:
from game import Game
from halma import Halma, FieldType
import pandas as pd
import strategies
import heuristics
import time
from main import print_board
import numpy as np

# Strategies and heuristics

In [2]:
strategies_list = [
    strategies.RandomStrategy,
    strategies.BestNextStateStrategy, 
    lambda h: strategies.MinimaxStrategy(h, 2), 
    lambda h: strategies.AlphaBetaStrategy(h, 2)]

heuristics_list = [
    heuristics.euclides_state_heuristic,
    heuristics.manhattan_state_heuristic,
    heuristics.manhattan_euclides_state_heuristic,
    heuristics.heatmap_heuristic,
    heuristics.adaptive_heatmap_heuristic,
]

In [3]:
def collect_game_data(player_white_strategy, player_black_strategy, stop_threshold):
    game = Game(player_white_strategy, player_black_strategy)
    move_times = {player : [] for player in Halma.players}

    while not game.halma.finished and game.halma.round < stop_threshold:
        times_list = move_times[game.current_player]
        
        start = time.time()
        game.next_move()
        end = time.time()

        times_list.append(end - start)
    
    game_time = sum(sum(player_times) for player_times in move_times.values())

    return (game.halma.winner, game.halma.round, move_times, game_time, game)

# Random vs best_move

In [5]:

df = pd.DataFrame()

for heuristic in heuristics_list:
    winner, rounds, times, game_time, _ = collect_game_data(strategies.RandomStrategy(), strategies.BestNextStateStrategy(heuristic), 400)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}

    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

df


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime
euclides_state_heuristic,PLAYER_BLACK,236.0,3.669927,0.000310,0.030791
manhattan_state_heuristic,PLAYER_BLACK,232.0,3.333832,0.000272,0.028468
manhattan_euclides_state_heuristic,PLAYER_BLACK,250.0,9.634362,0.000325,0.076750
heatmap_heuristic,PLAYER_BLACK,264.0,1.331427,0.000305,0.009782
adaptive_heatmap_heuristic,PLAYER_BLACK,260.0,1.689502,0.000402,0.012595


# Heuristics comparison, best_move strategy

In [6]:
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], []], names=[f"{player.name}_Heuristic" for player in Halma.players]))

for heuristic_1 in heuristics_list:
    for heuristic_2 in heuristics_list:
        winner, rounds, times, game_time, _ = collect_game_data(strategies.BestNextStateStrategy(heuristic_1), strategies.BestNextStateStrategy(heuristic_2), 400)

        time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}
        index = (heuristic_1.__name__, heuristic_2.__name__)
        df.loc[index, 'Winner'] = None if winner is None else winner.name
        df.loc[index, 'Rounds'] = rounds
        df.loc[index, 'Game Time'] = game_time

        for player, avg_time in time_averages.items():
            df.loc[index, f'{player.name}_AvgMoveTime'] = avg_time

df

Winner  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                             
euclides_state_heuristic           euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   
manhattan_state_heuristic          euclides_state_heuristic            PLAYER_WHITE   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic  PLAYER_WHITE   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
manhattan_euclides_state_heuristic euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   
heatmap_heuristic                  euclides_state_heuristic                    None   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic          None   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
adaptive_heatmap_heuristic         euclides_state_heuristic                    None   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic          None   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   

                                                                       Rounds  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                       
euclides_state_heuristic           euclides_state_heuristic             156.0   
                                   manhattan_state_heuristic            400.0   
                                   manhattan_euclides_state_heuristic   156.0   
                                   heatmap_heuristic                    400.0   
                                   adaptive_heatmap_heuristic           400.0   
manhattan_state_heuristic          euclides_state_heuristic             183.0   
                                   manhattan_state_heuristic            171.0   
                                   manhattan_euclides_state_heuristic   169.0   
                                   heatmap_heuristic                    199.0   
                                   adaptive_heatmap_heuristic           199.0   
manhattan_euclides_state_heuristic euclides_state_heuristic             156.0   
                                   manhattan_state_heuristic            400.0   
                                   manhattan_euclides_state_heuristic   156.0   
                                   heatmap_heuristic                    400.0   
                                   adaptive_heatmap_heuristic           400.0   
heatmap_heuristic                  euclides_state_heuristic             400.0   
                                   manhattan_state_heuristic            209.0   
                                   manhattan_euclides_state_heuristic   400.0   
                                   heatmap_heuristic                    179.0   
                          

# Best_move vs minimax depth = 2

In [16]:
df = pd.DataFrame()

for heuristic in heuristics_list:
    winner, rounds, times, game_time, _ = collect_game_data(strategies.BestNextStateStrategy(heuristic), strategies.MinimaxStrategy(heuristic, 2), 300)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}

    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

    print(f"{heuristic.__name__} DONE!")
df

euclides_state_heuristic DONE!
manhattan_state_heuristic DONE!
manhattan_euclides_state_heuristic DONE!
heatmap_heuristic DONE!
adaptive_heatmap_heuristic DONE!


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime
euclides_state_heuristic,PLAYER_BLACK,172.0,645.044368,0.041633,7.458883
manhattan_state_heuristic,PLAYER_BLACK,214.0,768.295393,0.041691,7.138640
manhattan_euclides_state_heuristic,PLAYER_BLACK,172.0,1225.321883,0.080464,14.167464
heatmap_heuristic,PLAYER_WHITE,237.0,220.086801,0.011243,1.853804
adaptive_heatmap_heuristic,None,300.0,246.133919,0.010898,1.640935


# Best_move vs alpha_beta depth = 2

In [5]:
df = pd.DataFrame()

for heuristic in heuristics_list:
    winner, rounds, times, game_time, _ = collect_game_data(strategies.BestNextStateStrategy(heuristic), strategies.AlphaBetaStrategy(heuristic, 2), 300)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}

    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

    print(f"{heuristic.__name__} DONE!")
df

euclides_state_heuristic DONE!
manhattan_state_heuristic DONE!
manhattan_euclides_state_heuristic DONE!
heatmap_heuristic DONE!
adaptive_heatmap_heuristic DONE!


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime
euclides_state_heuristic,PLAYER_BLACK,172.0,194.955740,0.035842,2.231086
manhattan_state_heuristic,PLAYER_BLACK,214.0,229.105206,0.030824,2.110346
manhattan_euclides_state_heuristic,PLAYER_BLACK,172.0,279.507546,0.079542,3.170546
heatmap_heuristic,PLAYER_WHITE,237.0,171.584682,0.011334,1.442677
adaptive_heatmap_heuristic,None,300.0,181.874745,0.010796,1.209768


# Heuristics comparison, alpha_beta depth = 2

In [18]:
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], []], names=[f"{player.name}_Heuristic" for player in Halma.players]))

for heuristic_1 in heuristics_list:
    for heuristic_2 in heuristics_list:
        winner, rounds, times, game_time, _ = collect_game_data(strategies.AlphaBetaStrategy(heuristic_1, 2), strategies.AlphaBetaStrategy(heuristic_2, 2), 300)

        time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}
        index = (heuristic_1.__name__, heuristic_2.__name__)
        df.loc[index, 'Winner'] = None if winner is None else winner.name
        df.loc[index, 'Rounds'] = rounds
        df.loc[index, 'Game Time'] = game_time

        for player, avg_time in time_averages.items():
            df.loc[index, f'{player.name}_AvgMoveTime'] = avg_time

df

Winner  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                             
euclides_state_heuristic           euclides_state_heuristic            PLAYER_WHITE   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
manhattan_state_heuristic          euclides_state_heuristic                    None   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
manhattan_euclides_state_heuristic euclides_state_heuristic            PLAYER_WHITE   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
heatmap_heuristic                  euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   
adaptive_heatmap_heuristic         euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   

                                                                       Rounds  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                       
euclides_state_heuristic           euclides_state_heuristic             215.0   
                                   manhattan_state_heuristic            275.0   
                                   manhattan_euclides_state_heuristic   208.0   
                                   heatmap_heuristic                    235.0   
                                   adaptive_heatmap_heuristic           235.0   
manhattan_state_heuristic          euclides_state_heuristic             300.0   
                                   manhattan_state_heuristic            300.0   
                                   manhattan_euclides_state_heuristic   254.0   
                                   heatmap_heuristic                    221.0   
                                   adaptive_heatmap_heuristic           221.0   
manhattan_euclides_state_heuristic euclides_state_heuristic             215.0   
                                   manhattan_state_heuristic            300.0   
                                   manhattan_euclides_state_heuristic   208.0   
                                   heatmap_heuristic                    217.0   
                                   adaptive_heatmap_heuristic           217.0   
heatmap_heuristic                  euclides_state_heuristic             202.0   
                                   manhattan_state_heuristic            300.0   
                                   manhattan_euclides_state_heuristic   176.0   
                                   heatmap_heuristic                    300.0   
                          

# Nodes visited comparison

In [4]:
df = pd.DataFrame()


for heuristic in heuristics_list:
    player_strategies = {
        FieldType.PLAYER_WHITE : strategies.MinimaxStrategy(heuristic, 2),
        FieldType.PLAYER_BLACK : strategies.AlphaBetaStrategy(heuristic, 2)
    }
    
    winner, rounds, times, game_time, _ = collect_game_data(player_strategies[FieldType.PLAYER_WHITE], player_strategies[FieldType.PLAYER_BLACK], 300)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}
    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

    for player, strategy in player_strategies.items():
        df.loc[heuristic.__name__, f'{player.name}_TotalNodesVisited'] = sum(strategy.nodes_visited)
        df.loc[heuristic.__name__, f'{player.name}_AvgNodesVisited'] = np.mean(np.array(strategy.nodes_visited))
    
    print(f"{heuristic.__name__} Done in {game_time}s")

df

euclides_state_heuristic Done in 1120.8246309757233s
manhattan_state_heuristic Done in 904.7901108264923s
manhattan_euclides_state_heuristic Done in 2139.110768556595s
heatmap_heuristic Done in 404.4942009449005s
adaptive_heatmap_heuristic Done in 448.92173647880554s


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime,PLAYER_WHITE_TotalNodesVisited,PLAYER_WHITE_AvgNodesVisited,PLAYER_BLACK_TotalNodesVisited,PLAYER_BLACK_AvgNodesVisited
euclides_state_heuristic,PLAYER_WHITE,215.0,1120.824631,7.705791,2.697189,3250827.0,30100.250000,390145.0,3646.214953
manhattan_state_heuristic,None,300.0,904.790111,4.159416,1.885085,2825907.0,18839.380000,652355.0,4378.221477
manhattan_euclides_state_heuristic,PLAYER_BLACK,208.0,2139.110769,16.744767,3.823606,3050499.0,29331.721154,384211.0,3694.336538
heatmap_heuristic,None,300.0,404.494201,1.537972,1.166432,2427722.0,16184.813333,496944.0,3335.194631
adaptive_heatmap_heuristic,None,300.0,448.921736,1.735108,1.266144,2497818.0,16652.120000,510652.0,3427.194631
